In [27]:
import pandas as pd
import numpy as np
import os
import gzip
import pickle
import re
import copy
from tqdm import tqdm
import time
import json
from datetime import datetime, timedelta
# pd.set_option('display.max_columns', 5000)
pd.set_option('display.max_rows', 100)

In [28]:
data = pd.read_csv('../5_processing_extracted_data/plotdata.csv') # date,row_index,hardware_name,TH/J,max_efficiency,year

# Convert the 'date' column to a datetime format
data['date'] = pd.to_datetime(data['date'])

# keep only data from 2011 onwards
data = data[data['date'] >= '2011-01-02']

# Create a 'month' column
data['month'] = (data['date']).dt.to_period('M')

# Group by 'month' and calculate the mean of 'TH/J'
quarterly_data = data.groupby('month')['TH/J'].mean().reset_index()

# Convert 'month' back to datetime (first day of the month)
quarterly_data['month'] = quarterly_data['month'].dt.to_timestamp()

quarterly_data.to_csv('monthly_efficiency.csv', index=False)

In [29]:
# Timestamp Graph,Intervals (Row),Network hashrate (TH/s),Bitcoins per block (BTC)
# 02/01/2009,1,4.97E-08,50
BitcoinData = pd.read_csv('BitcoinData.csv') 

df2 = pd.DataFrame(columns=['date', 'hashrate','coins_per_block'])

df2['date'] = pd.to_datetime(BitcoinData['Timestamp Graph'], format='%d/%m/%Y')
df2['hashrate'] = BitcoinData['Network hashrate (TH/s)']*1e12
df2['coins_per_block'] = BitcoinData['Bitcoins per block (BTC)']

df2 = df2.dropna()
df2 

,date,hashrate,coins_per_block
0,2009-01-02,4.970000e+04,50.00
1,2009-01-05,4.970000e+04,50.00
2,2009-01-08,6.960000e+05,50.00
3,2009-01-11,4.420000e+06,50.00
4,2009-01-14,6.310000e+06,50.00
...,...,...,...
1638,2022-06-18,2.350000e+20,6.25
1639,2022-06-21,2.260000e+20,6.25
1640,2022-06-24,1.900000e+20,6.25
1641,2022-06-27,2.000000e+20,6.25


In [30]:
hashrate = json.load(open('hashrate.json'))

# x	y
# 1231545600000	1.065220e-07
BitcoinData2 = pd.DataFrame(hashrate["hash-rate"])
BitcoinData2["date"] = pd.to_datetime(BitcoinData2["x"], unit='ms')
BitcoinData2["hashrate"] = BitcoinData2["y"]*1e12
BitcoinData2 = BitcoinData2.drop(columns=["x", "y"])

In [31]:
BitcoinData2

,date,hashrate
0,2009-01-10,1.065220e+05
1,2009-01-14,2.521021e+06
2,2009-01-18,5.588855e+06
3,2009-01-22,5.681174e+06
4,2009-01-26,6.270595e+06
...,...,...
1384,2024-03-08,5.928970e+20
1385,2024-03-12,6.294408e+20
1386,2024-03-16,5.963760e+20
1387,2024-03-20,6.010169e+20


In [32]:
temp = BitcoinData2[BitcoinData2["date"] > df2["date"].max()]
temp

,date,hashrate
1230,2022-07-01,2.146114e+20
1231,2022-07-05,2.179713e+20
1232,2022-07-09,2.061507e+20
1233,2022-07-13,2.059578e+20
1234,2022-07-17,1.968833e+20
...,...,...
1384,2024-03-08,5.928970e+20
1385,2024-03-12,6.294408e+20
1386,2024-03-16,5.963760e+20
1387,2024-03-20,6.010169e+20


In [33]:


# df2 is not defined for values above 2022
df2 = pd.concat([df2, temp])

# if block is nan, it should be 6.25
df2['coins_per_block'] = df2['coins_per_block'].fillna(6.25)

In [34]:
df2

,date,hashrate,coins_per_block
0,2009-01-02,4.970000e+04,50.00
1,2009-01-05,4.970000e+04,50.00
2,2009-01-08,6.960000e+05,50.00
3,2009-01-11,4.420000e+06,50.00
4,2009-01-14,6.310000e+06,50.00
...,...,...,...
1384,2024-03-08,5.928970e+20,6.25
1385,2024-03-12,6.294408e+20,6.25
1386,2024-03-16,5.963760e+20,6.25
1387,2024-03-20,6.010169e+20,6.25


In [35]:
# for each month in df2, get the monthly average and store it with the date being the first day of the month

# Create a 'month' column
from datetime import timedelta
df2['month'] = (df2['date']).dt.to_period('M')

# Group by 'month' and calculate the mean
df2 = df2.groupby('month')[["hashrate","coins_per_block"]].mean().reset_index()

# Convert 'month' back to datetime (first day of the month)
df2['date'] = df2['month'].dt.to_timestamp()
df2 = df2.drop(columns=["month"])
df2

,hashrate,coins_per_block,date
0,4.318540e+06,50.00,2009-01-01
1,6.009000e+06,50.00,2009-02-01
2,5.623000e+06,50.00,2009-03-01
3,5.712000e+06,50.00,2009-04-01
4,5.710000e+06,50.00,2009-05-01
...,...,...,...
178,4.722977e+20,6.25,2023-11-01
179,5.008642e+20,6.25,2023-12-01
180,5.166878e+20,6.25,2024-01-01
181,5.686831e+20,6.25,2024-02-01


In [36]:
# df2.to_csv('BitcoinData2.csv', index=False)

In [37]:
# Start,End,Open,High,Low,Close,Volume,Market Cap
# 2024-03-08,2024-03-09,66871.2,69451.15,66398.03,68202.87,71117263448.78049,1329444482209.993
price = pd.read_csv('../../pricehistory/price_full.csv') 



# Date,ln(P_max),Data monthly average,Model ln(P_eff)
# 2011-04-01,-12.485786548332777,1.6612383008356546e-06,1.5657608695648479e-06
efficiency = pd.read_csv('monthly_efficiency.csv') 

df1 = pd.DataFrame(columns=['date', 'price'])
df3 = pd.DataFrame(columns=['date', 'efficiency'])


df1['date'] = pd.to_datetime(price['Start'])
df1['price'] = price['Close']

df3['date'] = pd.to_datetime(efficiency['month'], format='%Y-%m-%d')
df3['efficiency'] = efficiency['TH/J']*1e12

joined = pd.merge(df1, df2, on='date', how='inner')
joined = pd.merge(joined, df3, on='date', how='inner')
joined = joined.dropna()

In [38]:
pd.set_option('display.max_rows', None)
joined.sample(10)

,date,price,hashrate,coins_per_block,efficiency
113,2014-05-01,457.760000,7.439091e+16,25.00,7.282989e+08
29,2021-05-01,57793.518868,1.592000e+20,6.25,1.534508e+10
124,2013-06-01,129.300000,1.490000e+14,25.00,4.497314e+08
129,2013-01-01,13.304100,2.175000e+13,25.00,1.166274e+08
11,2022-11-01,20481.554900,2.610995e+20,6.25,1.744762e+10
147,2011-07-01,15.397000,1.253636e+13,50.00,1.712089e+06
53,2019-05-01,5388.818933,5.153000e+19,12.50,9.433872e+09
4,2023-06-01,26806.980000,3.699255e+20,6.25,2.203848e+10
106,2014-12-01,379.366000,2.845000e+17,25.00,1.237387e+09
63,2018-07-01,6364.401842,3.866000e+19,12.50,8.366957e+09


In [39]:
# # joined["reward"] = joined["block"]*joined["price"]

# data = []

# for i in range(len(joined)):
#     date = joined['date'].iloc[i]
#     hashrate = joined['hashrate'].iloc[i]
#     efficiency = joined['efficiency'].iloc[i]
#     # reward = joined['reward'].iloc[i]
#     coins_per_block = joined['coins_per_block'].iloc[i]
#     price_per_coin = joined['price'].iloc[i]

#     aux = 2.0
#     electricity_cost = 0.05/(1000*3600)
#     constant = 6
#     watts = hashrate/efficiency
#     top = watts*aux*electricity_cost
#     coins_per_second = coins_per_block/600
#     bottom = constant*coins_per_second
#     cost_per_coin = top/bottom



#     joules_per_coin = watts/coins_per_block
#     joules_per_dollar_earned = joules_per_coin/price_per_coin

#     data.append([date, cost_per_coin, watts, coins_per_block, joules_per_coin,joules_per_dollar_earned])
    
#     # print(f"date: {date}")
#     # # print(f"hashrate: {hashrate}")
#     # # print(f"efficiency: {efficiency}")
#     # print(f"watts: {watts}")
#     # print(f"top: {top}")
#     # # print(f"reward: {reward}")
#     # print(f"bottom: {bottom}")
#     # print(f"cost: {cost_per_coin}")
#     # print("\n\n")

# df = pd.DataFrame(data, columns=['date', 'cost_per_coin', 'watts', 'coins_per_block', 'joules_per_coin','joules_per_dollar_earned'])

In [40]:
# joined = joined.drop("price", axis=1)
df = joined

In [41]:
# df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
# from datetime import timedelta
# df['avg'] = (df['date'] - timedelta(days=15)).dt.to_period('Q')
# df.sample(10)

In [42]:
# avg_data = df.groupby('avg')['cost'].mean().reset_index()
# avg_data

In [43]:
df['date'] = pd.to_datetime(df['date']).dt.date

In [44]:
df.sample(1)

,date,price,hashrate,coins_per_block,efficiency
24,2021-10-01,48080.372838,1.452000e+20,6.25,1.864925e+10


In [45]:
max_efficiency_df = pd.read_csv('../../hardwarelist/Bitcoin max updated2.csv', usecols=['date', 'max (TH/J)'])
max_efficiency_df['date'] = pd.to_datetime(max_efficiency_df['date']).dt.date
# Create a 'month' column
max_efficiency_df['month'] = pd.to_datetime(max_efficiency_df['date']).dt.to_period('M')
# Group by 'month' and calculate the mean of 'TH/J'
max_efficiency_df = max_efficiency_df.groupby('month')['max (TH/J)'].mean().reset_index()
# Convert 'month' back to datetime (first day of the month)
max_efficiency_df['date'] = max_efficiency_df['month'].dt.to_timestamp()
max_efficiency_df = max_efficiency_df.rename(columns={'max (TH/J)': 'max_efficiency'})
max_efficiency_df.drop(columns=["month"], inplace=True)
max_efficiency_df.tail(5)

,max_efficiency,date
164,0.0465,2022-09-01
165,0.0465,2022-10-01
166,0.0465,2022-11-01
167,0.0465,2022-12-01
168,0.0465,2023-01-01


In [46]:
general_df = pd.read_csv('../../bitcoinforum_general/5_processing_extracted_data/general.csv')
# keep only dates after 2011
general_df['date'] = pd.to_datetime(general_df['date']).dt.date
general_df = general_df[general_df['date'] >= datetime(2011, 1, 1).date()]
general_df.tail(5)

,date,speculation,adoption,altcoins,none,posts_count
166,2023-11-01,0.549451,0.376984,0.329115,0.342809,0.101586
167,2023-12-01,0.685197,0.280722,0.309755,0.365663,0.108028
168,2024-01-01,0.707851,0.315094,0.308360,0.299778,0.108523
169,2024-02-01,0.604396,0.291146,0.175528,0.441843,0.075818
170,2024-03-01,0.886700,0.031819,0.118027,0.573713,0.055996


In [47]:
# Merge with max efficiency
df['date'] = df['date'].astype(str)
max_efficiency_df['date'] = max_efficiency_df['date'].astype(str)
df = pd.merge(df, max_efficiency_df, on='date', how='left')

# Merge with general
# df['date'] = pd.to_datetime(df['date']).dt.date
# general_df['date'] = pd.to_datetime(general_df['date']).dt.date
df['date'] = df['date'].astype(str)
general_df['date'] = general_df['date'].astype(str)
df = pd.merge(df, general_df, on='date', how='left').fillna(method='ffill')

C:\Users\Timothe\AppData\Local\Temp\ipykernel_82424\787505272.py:11: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = pd.merge(df, general_df, on='date', how='left').fillna(method='ffill')


In [48]:
sentiment_df = pd.read_csv('../../bitcoinforum_general/5_processing_extracted_data/normalized_values_sentiment.csv')
sentiment_df['date'] = pd.to_datetime(sentiment_df['month']).dt.date
sentiment_df.drop(columns=["month","strongly negative","strongly positive"], inplace=True)
sentiment_df = sentiment_df[sentiment_df['date'] >= datetime(2011, 1, 1).date()]
sentiment_df['date'] = sentiment_df['date'].astype(str)
df = pd.merge(df, sentiment_df, on='date', how='left').fillna(method='ffill')

C:\Users\Timothe\AppData\Local\Temp\ipykernel_82424\3903061040.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = pd.merge(df, sentiment_df, on='date', how='left').fillna(method='ffill')


In [49]:
general_old_df = pd.read_csv('../../bitcoinforum_general/5_processing_extracted_data/general_old.csv')
general_old_df['date'] = pd.to_datetime(general_old_df['date']).dt.date
general_old_df = general_old_df[general_old_df['date'] >= datetime(2011, 1, 1).date()]
general_old_df.drop(columns=["posts_count","bitcoin_challenges","bitcoin_technology","bitcoin_adoption","educational_resources","altcoins"], inplace=True)
general_old_df['date'] = general_old_df['date'].astype(str)
general_old_df.tail(5)

,date,optimistic_speculation,pessimistic_speculation
166,2023-11-01,0.711246,0.127755
167,2023-12-01,0.678969,0.280989
168,2024-01-01,0.721154,0.298447
169,2024-02-01,0.717532,0.036391
170,2024-03-01,1.000000,0.377204


In [50]:
df = pd.merge(df, general_old_df, on='date', how='left').fillna(method='ffill')

C:\Users\Timothe\AppData\Local\Temp\ipykernel_82424\1782249016.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = pd.merge(df, general_old_df, on='date', how='left').fillna(method='ffill')


In [51]:
# Convert dates to end of the month
df['date'] = pd.to_datetime(df['date']) + pd.offsets.MonthEnd(0)

In [52]:
df.to_csv('monthly_stuff.csv', index=False)

In [6]:
import pandas as pd
pd.set_option('display.max_rows', 5000)
df = pd.read_csv('monthly_stuff.csv')
df["efficiency"] = df["efficiency"]/1e12
df[["date","efficiency","max_efficiency"]]

,date,efficiency,max_efficiency
0,2023-10-31,0.023880,NaN
1,2023-09-30,0.022223,NaN
2,2023-08-31,0.018209,NaN
3,2023-07-31,0.018302,NaN
4,2023-06-30,0.022038,NaN
5,2023-05-31,0.018392,NaN
6,2023-04-30,0.016545,NaN
7,2023-03-31,0.017792,NaN
8,2023-02-28,0.024040,NaN
9,2023-01-31,0.016595,0.046500
